# Group Analysis - FOOOFed EEG Analysis: Task

- Notes / ToDos: 
    - Check if difference in d's vs. difference in physiology (across loads)?
    
Overview:
- Words
- Words 
- Words

In [17]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from os import listdir
from os.path import join as pjoin

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import ttest_ind, sem, pearsonr


import pandas as pd

import patsy
import statsmodels.api as sm




from fooof import FOOOFGroup
from fooof.analysis import get_band_peak_group
from fooof.core.funcs import gaussian_function
from fooof.core.funcs import expo_nk_function

In [18]:
# Import custom code for this analysis
% autoreload 2
from io import *
from plts import *
from utils import *

# Settings

In [12]:
# Set path to load results from
res_path = '/Users/tom/Documents/Research/1-Projects/fooof/2-Data/Results/G2'

# Set indices to separate groups
from settings import YNG_INDS, OLD_INDS

# Set average function to use
avg_func = np.nanmean
#avg_func = np.nanmedian

# Wether to save out plots or not
save_fig = False

print('Number of young subjects:  ', len(YNG_INDS))
print('Number of  old  subjects:  ', len(OLD_INDS))

Number of young subjects:   17
Number of  old  subjects:   14


In [13]:
# Data settings
srate = 512
tmin, tmax = -0.85, 1.1
times = np.arange(tmin, tmax, 1/srate)
seg_times = [(-0.85, -0.35), (0.1, 0.6), (0.5, 1.0)]
n_subjs = 31

In [14]:
### Check dropped trials

In [15]:
# Load dropped trials & components
dropped_trials = np.load(pjoin(res_path, 'Group', 'dropped_trials.npy'))
dropped_components = np.load(pjoin(res_path, 'Group', 'dropped_components.npy'))

# Check dropped trials for each subject
print('SubNum \t\t # Dropped Trials \t # Dropped Components')
for ind, trials, components in zip(range(n_subjs), dropped_trials, dropped_components):
    temp_trials = trials[np.nonzero(trials)]
    temp_comps = components[np.nonzero(components)]
    print(ind, '\t\t', len(temp_trials), '\t\t\t', len(temp_comps))

SubNum 		 # Dropped Trials 	 # Dropped Components
0 		 5 			 3
1 		 15 			 4
2 		 1 			 0
3 		 13 			 1
4 		 18 			 2
5 		 0 			 7
6 		 28 			 6
7 		 7 			 0
8 		 29 			 4
9 		 4 			 4
10 		 2 			 3
11 		 0 			 4
12 		 14 			 7
13 		 2 			 2
14 		 2 			 3
15 		 0 			 5
16 		 3 			 3
17 		 11 			 0
18 		 2 			 2
19 		 109 			 3
20 		 38 			 6
21 		 14 			 4
22 		 0 			 2
23 		 0 			 4
24 		 1 			 3
25 		 15 			 0
26 		 4 			 3
27 		 12 			 4
28 		 41 			 4
29 		 34 			 5
30 		 0 			 3


## Group FOOOFing - Trial Averaged Data

- Ipsi & Contra

`all_alphas` & `all_slopes` have the shape [n_loads, n_subjs, n_times]

In [16]:
all_slopes, all_alphas = load_fooof_task(res_path)

# IT'S A MESS

In [ ]:
def all_areas(dat):
    outs = []
    for dd in dat:
        outs.append(area_gauss(dd[1], dd[2]))
    return np.array(outs)

def area_gauss(amp, bw):
    return (amp * bw/2) / 0.3989

In [ ]:
qq = np.apply_over_axes(area_gauss, all_alphas, axes)

#qq = np.nanmean(all_alphas, axis=0)
qq = all_alphas[1, :, :]
rr = np.nanmean(qq, axis=0)

In [ ]:
from scipy.stats import ttest_1samp

In [ ]:
ww = qq[:, 1] - qq[:, 0]

#ww[np.isnan(ww)] = 0

In [ ]:
ttest_1samp(ww[~np.isnan(ww)][OLD_INDS], 0)

In [ ]:
np.nanmedian(ww[YNG_INDS])

In [ ]:
np.nanmedian(ww[OLD_INDS])

In [ ]:
np.nanmedian(ww)

In [ ]:
ww[OLD_INDS]

In [ ]:
all_alphas[:, 12, :]

# Maybe there's some science here

Somewhere.

## Check Behaviour & Trial EPhys Corrs

- Baseline measures
- Task related 'reactivity' measures

Note: all_slopes & all_alphas have shape of [n_loads, n_subjs, n_times]

In [ ]:
# Average across analytic alpha measures to get canonical alpha measure

seg_masks = []
for seg in seg_times:
    seg_masks.append(np.logical_and(times >= seg[0], times <= seg[1]))

canalpha = np.zeros_like(all_alphas)
for subi, subj_dat in enumerate(canonical_group):
    for lodi in range(3):
        for segi, mask in enumerate(seg_masks):
            canalpha[lodi, subi, segi] = np.mean(subj_dat[lodi, mask])
            
#from scipy.cluster.vq import whiten
#wh_canalpha = whiten(canalpha)

In [ ]:
# Predict behaviour from 'absolute' FOOOF trial output measures
#   Note: as defined this model has three measures per subject, one for each load

# Settings
t_ind = 0

# Build the dataframe
df = pd.DataFrame()
df['slope'] = all_slopes[:, :, t_ind].flatten()

df['alpha'] = all_alphas[:, :, t_ind].flatten()
#df['alpha'] = canalpha[:, :, t_ind].flatten()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values

# Specify the model
#outcome, predictors = patsy.dmatrices("behav ~ slope + alpha + age", df)
outcome, predictors = patsy.dmatrices("behav ~ slope * alpha * age", df)
#outcome, predictors = patsy.dmatrices("behav ~ age + alpha + slope + alpha:age + slope:age", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# Predict behaviour from 'absolute' FOOOF trial output measures, separetely for each load

# Settings
t_ind = 0
l_ind = 1

# Build the dataframe
df = pd.DataFrame()
df['slope'] = all_slopes[l_ind, :, t_ind].flatten()
df['alpha'] = all_alphas[l_ind, :, t_ind].flatten()

df['behav'] = behav_dat[behav_dat['Load'] == l_ind+1]["d'"].values
df['age'] = behav_dat[behav_dat['Load'] == l_ind+1]["Age"].values

# Specift the model
#outcome, predictors = patsy.dmatrices("behav ~ slope + alpha + age", df)
#outcome, predictors = patsy.dmatrices("behav ~ slope * alpha * age", df)
#outcome, predictors = patsy.dmatrices("behav ~ age + alpha + slope + alpha:age + slope:age", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# Predict from absolute FOOOF results, split up by group

# Settings
group = 'young' # 'young', 'old'
group = 'old' 
t_ind = 1

inds = YNG_INDS if group == 'young' else OLD_INDS
age_gr = 1 if group == 'young' else 2

# Build the dataframe
df = pd.DataFrame()
df['slope'] = all_slopes[:, inds, t_ind].flatten()
df['alpha'] = all_alphas[:, inds, t_ind].flatten()
df['behav'] = behav_dat[behav_dat.Age == age_gr]["d'"].values

# 
outcome, predictors = patsy.dmatrices("behav ~ slope + alpha", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# Check relation between slope and alpha
#print(pearsonr(df.sl, df.al))
#plt.plot(df.sl, df.al, '.')

#print(pearsonr(df.beh, df.al))
#plt.plot(df.beh, df.al, '.')

#print(pearsonr(df.beh, df.sl))
#plt.plot(df.beh, df.sl, '.')

In [ ]:
# Check correlation between slope & d', and alpha & d' - for a specific load and time point

# Set which load and time point to use
l_ind = 0
t_ind = 0

print_stat("Sl-Lo-d'", *pearsonr(all_slopes[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == l_ind + 1]["d'"]))
print_stat("Al-Lo-d'", *pearsonr(all_alphas[l_ind, :, t_ind],
                                 behav_dat[behav_dat.Load == l_ind + 1]["d'"]))

In [ ]:
t_ind = 2

print_stat("Sl-d'", *pearsonr(all_slopes[:, :, t_ind].flatten(), behav_dat["d'"].values))
print_stat("Al-d'", *pearsonr(all_alphas[:, :, t_ind].flatten(), behav_dat["d'"].values))

In [ ]:
all_slopes.shape

In [ ]:
# Average d' with average measures

t_ind = 2

print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_slopes[:, :, t_ind], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, t_ind], 0)))

In [ ]:
# Average d' with average measures

t_ind = 2

print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_slopes[:, :, 2] - all_slopes[:, :, 0], 0)))
print_stat('XX', *pearsonr(avg_behav["d'"], np.mean(all_alphas[:, :, 2] - all_alphas[:, :, 0], 0)))

In [ ]:
#
sl_st_yng = np.vstack([all_slopes[0, YNG_INDS, :], all_slopes[1, YNG_INDS, :], all_slopes[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_slopes[0, OLD_INDS, :], all_slopes[1, OLD_INDS, :], all_slopes[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

#
canal_st_yng = np.vstack([canalpha[0, YNG_INDS, :], canalpha[1, YNG_INDS, :], canalpha[2, YNG_INDS, :]])
canal_st_old = np.vstack([canalpha[0, OLD_INDS, :], canalpha[1, OLD_INDS, :], canalpha[2, OLD_INDS, :]])

In [ ]:
ind = 2

print_stat('Yng-Sl', *pearsonr(sl_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Old-Sl', *pearsonr(sl_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

print('')

print_stat('Yng-Al', *pearsonr(al_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Old-Al', *pearsonr(al_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

print('')

print_stat('Yng-CanAl', *pearsonr(canal_st_yng[:, ind], behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Old-CanAl', *pearsonr(canal_st_old[:, ind], behav_dat[behav_dat.Age == 2]["d'"]))

In [ ]:
def calc_diff(dat, i1, i2):
    return dat[:, i1] - dat[:, i2]

In [ ]:
# Calculate difference measures
i1, i2 = 2, 0

sl_dif_yng = calc_diff(sl_st_yng, i1, i2)
sl_dif_old = calc_diff(sl_st_old, i1, i2)

al_dif_yng = calc_diff(al_st_yng, i1, i2)
al_dif_old = calc_diff(al_st_old, i1, i2)

canal_dif_yng = calc_diff(canal_st_yng, i1, i2)
canal_dif_old = calc_diff(canal_st_old, i1, i2)

In [ ]:
# Check correlations of difference measures with behaviour
print_stat('Sl-Yng', *pearsonr(sl_dif_yng, behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Sl-Old', *pearsonr(sl_dif_old, behav_dat[behav_dat.Age == 2]["d'"]))
print('')
print_stat('Al-Yng', *pearsonr(al_dif_yng, behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Al-Old', *pearsonr(al_dif_old, behav_dat[behav_dat.Age == 2]["d'"]))
print('')
print_stat('Al-Yng', *pearsonr(canal_dif_yng, behav_dat[behav_dat.Age == 1]["d'"]))
print_stat('Al-Old', *pearsonr(canal_dif_old, behav_dat[behav_dat.Age == 2]["d'"]))

In [ ]:
# Compare Canonical to FOOOF alpha

df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values

df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])
#df['al_dif'] = np.concatenate([canal_dif_yng, canal_dif_old])

# 
outcome, predictors = patsy.dmatrices("behav ~ al_dif * age", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# Build the dataframe

df = pd.DataFrame()

df['behav'] = behav_dat["d'"].values
df['age'] = behav_dat['Age'].values


df['sl_dif'] = np.concatenate([sl_dif_yng, sl_dif_old])
df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])
#df['al_dif'] = np.concatenate([canal_dif_yng, canal_dif_old])

# 
#outcome, predictors = patsy.dmatrices("behav ~ sl_dif + al_dif + age", df)
#outcome, predictors = patsy.dmatrices("behav ~ age * sl_dif", df)
outcome, predictors = patsy.dmatrices("behav ~ age * al_dif + sl_dif", df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()

# Check out the results
print(res.summary())

In [ ]:
# # Build the dataframe

# t_ind = 2
# l_ind = 0 

# df = pd.DataFrame()

# df['behav'] = behav_dat[behav_dat['Load'] == l_ind+1]["d'"].values
# df['age'] = behav_dat[behav_dat['Load'] == l_ind+1]["Age"].values

# df['sl_dif'] = np.concatenate([sl_dif_yng, sl_dif_old])
# df['al_dif'] = np.concatenate([al_dif_yng, al_dif_old])

# # 
# outcome, predictors = patsy.dmatrices("behav ~ sl_dif + al_dif + age", df)
# #outcome, predictors = patsy.dmatrices("behav ~ age * sl_dif", df)
# mod = sm.OLS(outcome, predictors)
# res = mod.fit()

# # Check out the results
# print(res.summary())

In [ ]:
#
al_ = np.mean(all_slopes[:, YNG_INDS, :], 0)

sl_st_yng = np.vstack([all_slopes[0, YNG_INDS, :], all_slopes[1, YNG_INDS, :], all_slopes[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_slopes[0, OLD_INDS, :], all_slopes[1, OLD_INDS, :], all_slopes[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

In [ ]:
print(pearsonr(sl_dif_yng, al_dif_yng))
print(pearsonr(sl_dif_old, al_dif_old))

In [ ]:
# Average across loads
sl_resps = np.diff(np.mean(all_slopes, 0))
al_resps = np.diff(np.mean(all_alphas, 0))

# # Take specific load
# load_ind = 2
# sl_resps = all_slopes[load_ind, :, :]
# al_resps = all_alphas[load_ind, :, :]

In [ ]:
# Correlations between reactivity measures
print_stat('SLR & ALR - 1', *pearsonr(sl_resps[:, 0], al_resps[:, 0]))
print_stat('SLR & ALR - 2', *pearsonr(sl_resps[:, 1], al_resps[:, 1]))

In [ ]:
edat = sl_resps; ind = 0;

print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == 1]["d'"], edat[YNG_INDS, ind]))
print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == 2]["d'"], edat[OLD_INDS, ind]))

In [ ]:
edat = sl_resps

print_stat('\tALL', *pearsonr(avg_behav["d'"], al_resps[:, 0]))
print_stat('\tYNG', *pearsonr(avg_behav[avg_behav['Age'] == 1]["d'"], np.mean(edat[YNG_INDS], 1)))
print_stat('\tOLD', *pearsonr(avg_behav[avg_behav['Age'] == 2]["d'"], np.mean(edat[OLD_INDS], 1)))

In [ ]:
print(pearsonr(avg_behav["CDA"], al_resps[:, 0]))
print(pearsonr(avg_behav["CDA"], sl_resps[:, 0]))

In [ ]:
# Get the change in behavioural scores between highes & lowest loads
delta_behavs = []
temp_df = behav_dat[behav_dat.Age == 1]
for subj_id in set(temp_df.SubjID.values):
    temp = behav_dat[behav_dat.SubjID == subj_id]
    delta_behavs.append(temp[temp.Load == 3]["d'"].values[0] - temp[temp.Load == 1]["d'"].values[0])
delta_behavs = np.array(delta_behavs)

In [ ]:
# Get the change in physiological scores between highes & lowest loads
temp_dat = all_slopes
temp = temp_dat[2, :, :] - temp_dat[0, :, :]
temp[np.isnan(temp)] = 0
delta_phys = temp[:, 2] - temp[:, 0]

In [ ]:
print('ALL:', pearsonr(delta_behavs, delta_phys[YNG_INDS]))
plt.scatter(delta_behavs, delta_phys[YNG_INDS])

In [ ]:
plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')
print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))
plt.legend()
plt.xlabel('Delta Behav');
plt.ylabel('Delta Physiology');

In [ ]:
#cur_dat = canalpha
#cur_dat = all_slopes
cur_dat = all_alphas

# Get phys-responsiveness, as late minus pre
temp = cur_dat[:, :, 2] - cur_dat[:, :, 1]
# Or - just take late
#temp = cur_dat[:, :, 2]

# Replace any nans as zeros
temp[np.isnan(temp)] = 0

# Get delta physiology as high load minus low load
delta_phys = temp[2, :] - temp[0, :]

In [ ]:
print('ALL:', pearsonr(delta_behavs, delta_phys))
print('OLD:', pearsonr(delta_behavs[OLD_INDS], delta_phys[OLD_INDS]))
print('YNG:', pearsonr(delta_behavs[YNG_INDS], delta_phys[YNG_INDS]))

plt.scatter(delta_behavs[OLD_INDS], delta_phys[OLD_INDS], label='OLD')
plt.scatter(delta_behavs[YNG_INDS], delta_phys[YNG_INDS], label='YNG')

#plt.ylim([-0.0000005, 0.0000005])

plt.legend()
plt.xlabel('Delta Behav');
plt.ylabel('Delta Physiology');

In [ ]:
#
sl_st_yng = np.vstack([all_slopes[0, YNG_INDS, :], all_slopes[1, YNG_INDS, :], all_slopes[2, YNG_INDS, :]])
sl_st_old = np.vstack([all_slopes[0, OLD_INDS, :], all_slopes[1, OLD_INDS, :], all_slopes[2, OLD_INDS, :]])

#
al_st_yng = np.vstack([all_alphas[0, YNG_INDS, :], all_alphas[1, YNG_INDS, :], all_alphas[2, YNG_INDS, :]])
al_st_old = np.vstack([all_alphas[0, OLD_INDS, :], all_alphas[1, OLD_INDS, :], all_alphas[2, OLD_INDS, :]])

#
canal_st_yng = np.vstack([canalpha[0, YNG_INDS, :], canalpha[1, YNG_INDS, :], canalpha[2, YNG_INDS, :]])
canal_st_old = np.vstack([canalpha[0, OLD_INDS, :], canalpha[1, OLD_INDS, :], canalpha[2, OLD_INDS, :]])